In [163]:
import PyPDF2
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import io
import os

In [82]:
def wrap_text(text, max_width, canvas, font_name="Helvetica", font_size=24):
    """Wraps the text to fit within the specified width."""
    words = text.split()
    lines = []
    line = ""
    
    for word in words:
        if canvas.stringWidth(line + " " + word, font_name, font_size) <= max_width:
            line += " " + word
        else:
            lines.append(line)
            line = word
    lines.append(line)
    
    return lines


In [81]:
def setup_and_draw_image(can, image_path):
    # Drawing and setup for image
    image_width = 1792
    image_height = 1024
    pdf_width, pdf_height = letter
    scale_width = pdf_width / image_width
    scale_height = pdf_height / image_height
    scale = min(scale_width, scale_height)
    new_image_width = image_width * scale
    new_image_height = image_height * scale
    x = (pdf_width - new_image_width) / 2
    y = (pdf_height - new_image_height) / 2
    can.drawImage(image_path, x, y, new_image_width, new_image_height)
    
    return x, y, new_image_width, new_image_height

In [137]:
def draw_text_on_pdf(can, text1, text2):
    pdf_width, pdf_height = letter

    # Set up for Text 1
    font_size = 30  # adjust this for desired font size
    text_width = pdf_width - 200  # leaving some margins
    can.setFont("Times-Roman", font_size)

    # Calculate the space required for wrapped Text1
    wrapped_text1 = wrap_text(text1, text_width, can)
    space_required_for_text1 = len(wrapped_text1) * (font_size + 5)

    # Start drawing Text 1 from top, leaving some margin
    text1_y = pdf_height - 80  # Starting near the top, adjust as needed
    for line in wrapped_text1:
        can.drawString((pdf_width - can.stringWidth(line, "Times-Roman", font_size)) / 2, text1_y, line)
        text1_y -= (font_size + 5)

    # Set up for Text 2
    wrapped_text2 = wrap_text(text2, text_width, can)
    space_required_for_text2 = len(wrapped_text2) * (font_size + 5)

    # Start drawing Text 2 from bottom, leaving some margin
    text2_y = -150 + space_required_for_text2  # adjust the '50' as needed
    for line in reversed(wrapped_text2):  # reversed to print from bottom up
        can.drawString((pdf_width - can.stringWidth(line, "Times-Roman", font_size)) / 2, text2_y, line)
        text2_y += (font_size + 5)


In [138]:
def create_pdf_with_text(image_path, text1, text2):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)
    x, y, new_image_width, new_image_height = setup_and_draw_image(can, image_path)
    draw_text_on_pdf(can, text1=text1, text2=text2)
    can.showPage()
    can.save()
    packet.seek(0)
    # Save the resulting PDF to a file
    new_pdf = PyPDF2.PdfReader(packet)
    pdf = PyPDF2.PdfWriter()
    pdf.add_page(new_pdf.pages[0])
    with open('output.pdf', 'wb') as out:
        pdf.write(out)


In [173]:

# Define the image path and the two texts
image_path = 'Nunta.png'
base_text1 = "{} imi face mare placere sa te invit la nunta mea."
text2 = "Dupa cum stii m-am indragostit de curand si am ales sa imi petrec restul vietii cu ea. Sper ca vei fi alaturi de mine in aceasta zi speciala. Aleasa mea e ma-ta!"

output_directory = r'C:\Users\ediacon\OneDrive - MORNINGSTAR INC\Documents\Python Scripts\02.100 Days of Python\24.Day_24\Output'


In [182]:
with open("List_of_names.txt",'r') as nume:
    list_of_names = [x.strip() for x in nume.readlines()]

In [184]:
for name in list_of_names:
    text1 = base_text1.format(name)
    create_pdf_with_text(image_path, text1, text2)
    
    
    output_path = os.path.join(output_directory, f"{name}.pdf")
    os.rename('output.pdf', output_path)

In [183]:
list_of_names

['Daniel']